In [ ]:
import torch.nn as nn
from transformers import AutoModelForCausalLM

# Carrega o modelo pré-treinado
#model = AutoModel.from_pretrained('meta-llama/Llama-3.2-3B')

model = AutoModelForCausalLM.from_pretrained(
    'unsloth/Llama-3.2-1B',
    device_map='auto',  # Faz com que o modelo seja carregado parcialmente
    offload_folder="offload",  # Pasta para descarregar partes do modelo para o disco
    offload_state_dict=True  # Também descarrega os parâmetros de estado para o disco
)
print(model)

config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [ ]:
# Acessa a parte 'model' do LlamaForCausalLM
llama_model = model.model

# Lista dos atributos a serem removidos, exceto 'embed_tokens'
atributos_para_remover = [attr for attr in llama_model._modules if attr != 'embed_tokens' and attr != 'rotary_emb']

for attr in atributos_para_remover:
    delattr(llama_model, attr)

# Opcional: remover também 'lm_head' do modelo principal
if hasattr(model, 'lm_head'):
    delattr(model, 'lm_head')

# Remove todas as camadas de decodificação
llama_model.layers = nn.ModuleList()

model.config.num_hidden_layers = 0

# Verifica a estrutura do modelo após a modificação
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (rotary_emb): LlamaRotaryEmbedding()
    (layers): ModuleList()
  )
)


In [ ]:
import os

# Define o diretório onde o modelo será salvo
diretorio_salvar = './Llama-3.2-1b-Gather'

# Cria o diretório se ele não existir
os.makedirs(diretorio_salvar, exist_ok=True)

# Salva o modelo e a configuração
model.save_pretrained(diretorio_salvar)

print(f"Modelo salvo em: {diretorio_salvar}")

Modelo salvo em: ./Llama-3.2-1b-Gather
